In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

INPUT_DIR = '../data'
OUTPUT_DIR = 'outputs/'
SEED = 1230
# 初期設定
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED) # NOTE ソートのアルゴリズムがクイックソートのため、固定しないと順位が変わる可能性あり

In [2]:
# ログデータ
train_log = pd.read_csv(os.path.join(INPUT_DIR, 'train_log.csv'),)
train_label = pd.read_csv(os.path.join(INPUT_DIR, 'train_label.csv'),)
test_log = pd.read_csv(os.path.join(INPUT_DIR, 'test_log.csv'),)
# 宿データ
yado_df = pd.read_csv(os.path.join(INPUT_DIR, "yado.csv"))

In [3]:
def create_co_matrix(df: pd.DataFrame, df_: pd.DataFrame):
    df = df.merge(df_, how="inner", on="session_id")
    df = df.groupby(["yad_no_x", "yad_no_y"]).size().reset_index()
    df.columns = ["yad_no", "candidate_yad_no", "cnt"]
    return df

In [4]:
train_whole_df = pd.concat([train_log, train_label], axis=0, ignore_index=True,)
# 予約も含めた、閲覧共起行列の作成
visit_co_df_train = create_co_matrix(train_whole_df, train_whole_df)
visit_co_df_train.rename(columns={"cnt": "visit_cnt_train"}, inplace=True)

# テストの閲覧共起
visit_co_df_test = create_co_matrix(test_log, test_log)
visit_co_df_test.rename(columns={"cnt": "visit_cnt_test"}, inplace=True)

# trainの予約共起
reserve_co_df = create_co_matrix(train_log, train_label)
reserve_co_df.rename(columns={"cnt": "reserve_cnt"}, inplace=True)

In [5]:
RESERVE_STRENGTH = 10
TEST_CO_STRENGTH = 1 # NOTE 30以上であればほとんど変わらない？
# 学習データとテストデータの共起を外部結合
co_df = visit_co_df_train.merge(visit_co_df_test, how="outer",on=["yad_no", "candidate_yad_no"]).fillna(0)
# 予約結合
co_df = co_df.merge(reserve_co_df, how="left", on=["yad_no", "candidate_yad_no"]).fillna(0)
# 重みづけ
co_df["score"] = co_df["visit_cnt_train"] + co_df["reserve_cnt"] * RESERVE_STRENGTH # TODO 分布の値を用いて変動的にしたい
co_df["score"] += co_df["visit_cnt_test"] * TEST_CO_STRENGTH # TODO 分布の値を用いて変動的にしたい
# SELECT
co_df = co_df[["yad_no", "candidate_yad_no", "score"]]

In [6]:
COL = "sml_cd" # TODO ここ雑
additional_co_df = co_df.query("yad_no == candidate_yad_no").drop(columns=["candidate_yad_no"])
# 地域情報紐付け
additional_co_df = additional_co_df.merge(yado_df[["yad_no", COL]])
# 地域ごとのスコアでランクづけ
additional_co_df[f"{COL}_rank"] = additional_co_df.groupby(COL).score.rank(method="first", ascending=False).astype(int)
additional_co_df = additional_co_df.query(f"{COL}_rank < 10")
additional_co_df = yado_df[["yad_no", COL]].merge(additional_co_df, how="left", on=COL)
additional_co_df.rename(columns={"yad_no_x": "yad_no", "yad_no_y": "candidate_yad_no"}, inplace=True)
additional_co_df["score"] = 10 - additional_co_df[f"{COL}_rank"]
# SELECT
additional_co_df = additional_co_df[["yad_no", "candidate_yad_no", "score"]]

In [7]:
co_df

,yad_no,candidate_yad_no,score
0,1,1,31.0
1,1,6431,1.0
2,2,2,91.0
3,2,36,12.0
4,2,217,11.0
...,...,...,...
266807,13798,13438,2.0
266808,13804,2983,1.0
266809,13804,4226,3.0
266810,13804,12835,1.0


In [8]:
# 最終的に使用するスコア行列
final_co_df = pd.concat([co_df, additional_co_df], axis=0).groupby(["yad_no", "candidate_yad_no"]).score.max().reset_index()

In [9]:
# セッション内閲覧は候補より上位にするため
SEQ_NO_STRENGTH = final_co_df.score.max()

def predict(df: pd.DataFrame) -> pd.DataFrame:
    # 後で最後の閲覧を消すため
    last_views = df.groupby("session_id").yad_no.last().rename("last_view_yad_no")
    # 後でセッション内の閲覧を上位にするため
    unique_views = df.groupby(["session_id", "yad_no"]).seq_no.min().reset_index()
    # スコア行列マージ
    df_ = df.merge(final_co_df, how="left", on="yad_no")
    # ランキング
    df_ = df_.groupby(["session_id", "candidate_yad_no"]).score.sum().reset_index()
    df_["score_rank"] = df_.groupby("session_id").score.rank(ascending=False, method="first")
    # スコアTOP15に絞る（計算量減らすため）
    df_ = df_.query("score_rank <= 15")
    # セッション内を上位に
    df_ = df_.merge(unique_views, how="left", left_on=["session_id", "candidate_yad_no"], right_on=["session_id", "yad_no"])
    df_.loc[df_.seq_no.notnull(), "score"] += SEQ_NO_STRENGTH 
    # 最後に閲覧の宿を消す
    df_ = df_.merge(last_views, how="left", on="session_id")
    df_ = df_.query("candidate_yad_no != last_view_yad_no")
    # 最終ランキング
    df_["score_rank"] = df_.groupby("session_id").score.rank(ascending=False, method="first")
    # TOP10を提示
    df_ = df_.query("score_rank <= 10").groupby("session_id").head(10)
    # submit形式に修正
    return df_.pivot_table(index="session_id", columns="score_rank", values="candidate_yad_no").fillna(0).astype(int)

In [10]:
test_pred = predict(test_log)

In [11]:
col_names = ['predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8', 'predict_9']
sample_submission = pd.DataFrame(test_pred.values.tolist(), columns=col_names)
sample_submission

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,3560,4714,11561,2680,4545,9534,4420,5466,9830,6563
1,143,4066,6555,7014,7913,12862,11923,8108,11237,6129
2,757,9190,7710,9910,410,1774,10485,6721,3400,6730
3,12341,3359,6991,5080,13521,1542,10746,2795,4180,5657
4,9020,2862,10826,12029,3476,3844,3854,5372,6161,9623
...,...,...,...,...,...,...,...,...,...,...
174695,1997,7888,11123,1885,8771,7641,831,5744,7062,10997
174696,12432,13220,3802,1227,899,4014,2164,3644,13702,448
174697,13241,13797,2087,7308,8143,12939,13719,844,5810,7379
174698,3100,10287,2692,2373,3002,4976,12281,5513,3096,5079


In [12]:
(sample_submission.values == 0).sum() # NOTE 0がある、勿体無い

7812

In [13]:
sample_submission.to_csv(os.path.join(OUTPUT_DIR, "sample_submission_e002.csv"), index=False)